In [ ]:
using JSON, SDDP,DataFrames
using Plots, StatPlots
const mm = Plots.mm
const pt = Plots.pt
fntsm = Plots.font("times", 10.0pt)
fntlg = Plots.font("times", 12.0pt)
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm,left_margin=10mm,bottom_margin=7.5mm)
default(size=(800,600),top_margin=0mm, right_margin=0mm)
gr()

In [ ]:
# Plot main figure
function getresults(results, key, scalefactor=1.0)
    Q = [0.01, 0.1, 0.5, 0.9, 0.99, 0.01, 0.99]
    g2 = hcat([scalefactor.*r[key] for r in results]...)
    g3 = hcat([quantile(g2[g,:], Q) for g in 1:size(g2, 1)]...)'
    DataFrame(week=1:size(g3, 1), x10=g3[:,1], x25=g3[:,2], x50=g3[:,3], x75=g3[:,4], x90=g3[:,5],x05=g3[:,6],x95=g3[:,7])
end
function ribbonplot!(data, xkey, lkey, ukey, c, fillalpha)
    x = data[xkey]
    yl = 0.5 * (data[lkey] + data[ukey]) #blue[:x10]
    yu = data[ukey] - yl #blue[:x90] - yl
    plot!(x, yl, ribbon=yu, c=c, fillalpha=fillalpha, alpha=0)
end

function plotpowder(data, key, ylims, ylabel,title,scalefactor=1.0;plotline::Int=0)
    blue = getresults(data, key, scalefactor)
    plot()
    ribbonplot!(blue, :week, :x10, :x90, "#00467F", 0.3)
    ribbonplot!(blue, :week, :x25, :x75, "#00467F", 0.3)
    @df blue plot!(:week, :x50, c="#00467F", w=2)
    if plotline > 0
        plot!(scalefactor*data[plotline][key], c="red", w=3)
    end
    xticks = collect(1:8.66:52)
    xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
    plot!(legend=false, ylims=ylims, ylabel=ylabel,title=title, xlims=(1,52), xticks=(xticks, xticklabels), xlabel="")
    plot!(size=(500,300))
end

function plotall(data; plotprice=false,plotline=0)
    T = 52
    for d in data
        d[:contracted_proportion] = fill(0.0, T)
        d[:milk_production] = fill(0.0, T)
        d[:milk_sold] = fill(0.0, T)
        for t in 1:T
            d[:milk_production][t] = sum(d[:milk][1:t]) 
            d[:milk_sold][t] = sum(d[:milk_sales][1:t]) 
            d[:contracted_proportion][t] = d[:milk_sold][t] ./ sum(d[:milk])
        end
    end
    if !plotprice
        plot(
            plotpowder(data, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(a)",plotline=plotline),
            plotpowder(data, :P, (0,3500), "Pasture Cover\n(kg/Ha)","(b)",plotline=plotline),
            plotpowder(data, :b, (0,6), "Palm Kernel Fed\n(kg/Cow/Day)", "(c)", 1 / 3 / 7,plotline=plotline),
            plotpowder(data, :milk_production, (0,2000), "Milk Production\n(kg)", "(d)",plotline=plotline),
            plotpowder(data, :milk_sold, (0,2000), "Milk Sales\n(kg)", "(e)",plotline=plotline),
            plotpowder(data, :contracted_proportion, (0, 1.25), "Proportion of Milk Contracted\n(fraction)", "(f)", plotline=plotline),
            layout=(2,3), size=(1500,600)
        )
    else
        parameters = JSON.parsefile("model.parameters.json")
        function futures_contribution(gt, w, auction)
            y = 0.0
            for i in (auction+1):length(w)
                y += w[i] * (0.96 ^ (i - auction) * gt + 0.252 * sum(0.96^(j-1) for j in 1:(i-auction)))
            end
            y
        end
        for n in data
            n[:gdt] = [p[1] for p in n[:price]]
            n[:accumulated] = [p[2] for p in n[:price]]
            n[:futures] = zeros(Float64, T)
            for stage in 1:T
                auction = findlast(x->x<=stage, parameters["auction_weeks"])
                n[:futures][stage] = n[:accumulated][stage] + futures_contribution(n[:gdt][stage], parameters["sales_curve"], auction)
            end
        end
        plot(
            plotpowder(data, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(a)",plotline=plotline),
            plotpowder(data, :P, (0,3500), "Pasture Cover\n(kg/Ha)","(b)",plotline=plotline),
            plotpowder(data, :b, (0,6), "Palm Kernel Fed\n(kg/Cow/Day)", "(c)", 1 / 3 / 7,plotline=plotline),
            plotpowder(data, :milk_production, (0,2000), "Milk Production\n(kg)", "(d)",plotline=plotline),
            plotpowder(data, :milk_sold, (0,2000), "Milk Sales\n(kg)", "(e)",plotline=plotline),
            plotpowder(data, :contracted_proportion, (0, 1.25), "Proportion of Milk Contracted\n(fraction)", "(f)", plotline=plotline),
            plotpowder(data, :gdt, (0, 10), "Spot Price\n(\$/kg)", "(h)",plotline=plotline),
            plotpowder(data, :accumulated, (0, 10), "Accumulated Price\n(\$/kg)", "(h)",plotline=plotline),
            plotpowder(data, :futures, (0, 10), "Futures Price\n(\$/kg)", "(i)",plotline=plotline),
            layout=(3,3), size=(1500,900)
        )
    end
end

function plotdensity!(data, label, color, w=1, a=1.0, densityflag=true)
    x = [sim[:objective] - 3536 for sim in data]
    if densityflag
        density!(x, label=label, c=color, w=w, alpha=a)
    else
        vline!([mean(x)], label="", c=color, w=w, alpha=a, linestyle=:dash)
    end
end
function addweather!(dataset, params)
    for n in dataset
        n[:rainfall] = [params["niwa_data"][t][n[:noise][t]]["rainfall"] for t in 1:52]
        n[:evapotranspiration] = [params["niwa_data"][t][n[:noise][t]]["evapotranspiration"] for t in 1:52]
    end
end

In [ ]:
modelone(s) = joinpath("model_one", s)
neutral = SDDP.load(modelone("NeutralWithContracting.results"))[1:2000];
params = JSON.parsefile(modelone("neutral_with_contracting.json"))
addweather!(neutral, params)
println("Noise")

plot(
plotpowder(neutral, :price, (2, 10), "Milk Price Price\n(\$/kg)", "(a)"),
plotpowder(neutral, :evapotranspiration, (0, 50), "Evapotranspiration\n(mm/Week)", "(b)"),
plotpowder(neutral, :rainfall, (0, 300), "Rainfall\n(mm/Week)", "(c)"),
    layout=(1,3), size=(1500,300)
)
savefig(modelone("model_one_noise.pdf"))

println("Neutral")
plotall(neutral)
savefig(modelone("neutral_no_selling.pdf"))

println("Averse")
averse = SDDP.load(modelone("AverseNoContracting.results"))[1:2000];
addweather!(averse, params)
plotall(averse)
savefig(modelone("averse_no_selling.pdf"))

println("Selling")
averse_sell = SDDP.load(modelone("AverseWithContracting.results"))[1:2000];
addweather!(averse_sell, params)
plotall(averse_sell)
savefig(modelone("averse_selling.pdf"))

println("High")
high_averse_sell = SDDP.load(modelone("HighAverseWithContracting.results"))[1:2000];
plotall(high_averse_sell)
savefig(modelone("high_averse_selling.pdf"))

println("Low")
low_averse_sell = SDDP.load(modelone("LowAverseWithContracting.results"))[1:2000];
plotall(low_averse_sell)
savefig(modelone("low_averse_selling.pdf"))

println("Density")
scale = 1.5
plot(
    xlabel="Operating Profit (\$/Ha)\n",
    ylabel="Smoothed Density",
    ytick=false,
    right_margin=5mm, bottom_margin=10mm, 
    size=(scale*500,scale*300),
    legend=:topright, 
    grid=false
)

for (data, key, colour, alpha) in [
        (neutral, "Risk Neutral", "#00467F", 0.5),
        (averse, "Medium Risk Averse", "#00467F", 1.0),
        (low_averse_sell, "Low Risk Aversion", "red", 0.4),
        (averse_sell, "Medium Risk Aversion", "red", 0.7),
        (high_averse_sell, "High Risk Aversion", "red", 1.0)
    ]
    plotdensity!(data, key, colour,3, alpha, true)
    plotdensity!(data, "", colour,3, alpha, false)
    plot!(xlims=(-3000,9000))
end
savefig(modelone("density_comparison.pdf"))

println("Heuristic")
plot(
    xlabel="Operating Profit (\$/Ha)\n",
    ylabel="Smoothed Density",
    ytick=false,
    right_margin=5mm, bottom_margin=10mm, 
    size=(scale*500,scale*300),
    legend=:topright, 
    grid=false
)
plotdensity!(neutral, "Neutral", "#00467F",3, 1.0, true)
plotdensity!(neutral, "", "#00467F",1, 1.0, false)
plotdensity!(low_averse_sell, "Risk Averse", "red",3, 1.0, true)
plotdensity!(low_averse_sell, "Risk Averse", "red",1, 1.0, false)
heuristic = deepcopy(neutral)
for h in heuristic
    h[:objective] += 1000 * (6.255 - 0.015 - h[:price][end])
end
plotdensity!(heuristic, "Heuristic", "green",3, 1.0, true)
plotdensity!(heuristic, "", "green",1, 1.0, false)
savefig(modelone("heuristic.pdf"))

In [ ]:
using JuMP, Clp
function recalculate_objective(dataset)
    objectives = [
        (sum(a[:milk]) * clamp(a[:price][end], 3, 9) -
            0.5 * sum(a[:b]) -
            0.5 * sum(a[:i]) -
            0.275 * sum(a[:h]) -
            sum(x[1] for x in a[:Δ]) -
            1e2 * sum(x[2] for x in a[:Δ]) +
            0.0001 * sum(a[:W]) -
            3536
        ) for a in dataset
    ]
end
function hedge(profits, observations, quantity)
    profits + quantity * (mean(observations) - observations)
end
cvar(x::Vector{Float64}, b::Float64) = mean(x[x .<= quantile(x, b)])
function cvar_hedgeratio(profits, observations, beta)
    J = length(profits)
    m = Model(solver=ClpSolver())
    @variables(m, begin
        ξ
        h
        z[1:J] >= 0
    end)
    @objective(m, Min, ξ + sum(z) / beta / J)
    @constraint(m, z .>= -hedge(profits, observations, h) - ξ )
    solve(m)
    getvalue(h)
end
function minvar_hedgeratio(profit, observations)
    cov(profit, observations) / var(observations)
end

function plottransformeddistribution(dataset; ylims=(-2000,6000), titlestring="", titlecode="", kwargs...) 
    objectives = recalculate_objective(dataset)
    prices = [clamp(x[:price][end], 3, 9) for x in dataset]
    
    Q = linspace(0, minvar_hedgeratio(objectives, prices) * 2, 100)
    profits = [hedge(objectives, prices, q) for q in Q]
    data = hcat([
                quantile(p, [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])
            for p in profits]...)'
    
    μ = mean.(profits)
    
    standard_deviation = std.(profits)
    var_hedge_ratio = minvar_hedgeratio(objectives, prices)
    plot_variance = true
    
    α = 0.1
    average_value_at_risk = cvar.(profits, α)
    cvar_hedge_ratio = cvar_hedgeratio(objectives, prices, α)
    plot_cvar = false
    
    pltr = plot(
            legend=false,
            xlabel="Contracted Amount (kg)\n",
            ylabel="Operating Profit (\$/Ha)",
            left_margin=5mm,
            right_margin=5mm,
            bottom_margin=10mm,
            top_margin=12.5mm,
            ylims=ylims,
            title="$(titlestring)\n \n($(titlecode))",
            size=(500,300);
            kwargs...
            )
    plot!(Q, data, c="black", w=1, linestyle=:dot)
    plot!(Q, μ, c="black", w=3)
    
    if plot_cvar
        plot!(Q, average_value_at_risk, c="orange", w=3, linestyle=:dash)
        vline!([cvar_hedge_ratio], c="orange", w=2)
    end
    if plot_variance
        plot!(Q, standard_deviation, c="red", w=3, linestyle=:dash)
        vline!([var_hedge_ratio], c="red", w=2)
    end
        
    pltl = plot(
        legend=false,
        xlabel="Milk Price (\$/kg)\n",
        ylabel="Operating Profit (\$/Ha)",
        title="($(titlecode+3))",
        left_margin=5mm,
        right_margin=5mm,
        bottom_margin=10mm,
        ylims=ylims,
        size=(500,300);
        kwargs...
    )
    
    idx = quantile(objectives, 0.01) .<= objectives .<= quantile(objectives, 0.99)
    myscatter!(x, y, c) = scatter!(x, y, c=c, alpha=0.2, markersize=3, markerstrokewidth=0)
    
    myscatter!(prices[idx], objectives[idx],"#00467F")
    if plot_variance
        hedged_profits = hedge(objectives[idx], prices[idx], var_hedge_ratio)
        myscatter!(prices[idx],hedged_profits,"red")
    end
    if plot_cvar
        hedged_profits = hedge(objectives[idx], prices[idx], cvar_hedge_ratio)
        myscatter!(prices[idx],hedged_profits, "orange")
    end
    plot(pltr, pltl, layout=(2,1))
end

plot(
    plottransformeddistribution(neutral,#f,
        titlecode='a', titlestring="Risk Neutral"),
    plottransformeddistribution(averse,# f,
        titlecode='b', titlestring="Risk Averse: No Contracting"),
    plottransformeddistribution(averse_sell,# f,
        titlecode='c', titlestring="Risk Averse: Contracting"),
    layout=(1, 3), size=(1500,700)
)
savefig(modelone("heuristic_distribution.pdf"))

In [ ]:
function plottransformeddistribution2(dataset; ylims=(1000,3500), titlestring="", titlecode="", kwargs...) 
    prices = [clamp(x[:price][end], 3, 9) for x in dataset]
    
    unhedged_returns = recalculate_objective(dataset)
    objectives = hedge(unhedged_returns, prices, minvar_hedgeratio(unhedged_returns, prices))
    
    rainfall = [sum(x[:rainfall][21:30]) for x in dataset]
    rainfall = log.(rainfall)
    Q = linspace(0, minvar_hedgeratio(objectives, rainfall) * 2, 100)
#     Q = linspace(0, 1000# , 100)
    profits = [hedge(objectives, rainfall, q) for q in Q]
    
    data = hcat([
                quantile(p, [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])
            for p in profits]...)'
    
    μ = mean.(profits)
    
    standard_deviation = std.(profits)
    @show var_hedge_ratio = minvar_hedgeratio(objectives, rainfall)
    plot_variance = true
    
    pltr = plot(
            legend=false,
            xlabel="Contracted Amount (Contracts)\n",
            ylabel="Operating Profit (\$/Ha)",
            left_margin=5mm,
            right_margin=5mm,
            bottom_margin=10mm,
            top_margin=12.5mm,
            ylims=ylims,
            title="$(titlestring)\n \n($(titlecode))",
            size=(500,300);
            kwargs...
            )
    plot!(Q, data, c="black", w=1, linestyle=:dot)
    plot!(Q, μ, c="black", w=3)
    
    plot!(Q, standard_deviation, c="red", w=3, linestyle=:dash)
    vline!([var_hedge_ratio], c="red", w=2)
        
    pltl = plot(
        legend=false,
        xlabel="Log Rainfall\n",
        ylabel="Operating Profit (\$/Ha)",
        title="($(titlecode+3))",
        left_margin=5mm,
        right_margin=5mm,
        bottom_margin=10mm,
        ylims=ylims,
        size=(500,300);
        kwargs...
    )
    
    idx = quantile(objectives, 0.01) .<= objectives .<= quantile(objectives, 0.99)
    idx = fill(true, length(objectives))
    myscatter!(x, y, c) = scatter!(x, y, c=c, alpha=0.2, markersize=3, markerstrokewidth=0)
    
#     myscatter!(exp.(rainfall[idx]), objectives[idx],"#00467F")
    myscatter!(rainfall[idx], objectives[idx],"#00467F")
    hedged_returns = hedge(objectives[idx], rainfall[idx], var_hedge_ratio)
    @show std(hedged_returns)
#     myscatter!(exp.(rainfall[idx]),hedged_returns,"red")
    myscatter!(rainfall[idx],hedged_returns,"red")
    plt = plot(pltr, pltl, layout=(2,1))
    
    
    ev = [sum(x[:evapotranspiration]) for x in dataset]
    ev_hedged = hedge(objectives, ev, minvar_hedgeratio(objectives, ev))
    
    gr = [sum(x[:gr]) for x in dataset]
    gr_hedged = hedge(objectives, gr, minvar_hedgeratio(objectives, gr))
    
    plot(
        size = (750, 450),
        yticks=false,
        xlabel="Operating Profit (\$/Ha)\n",
        ylabel="Smoothed Density",
        left_margin=2mm,
        ylims=(0, 0.002),
        legend=:topleft
    )
    density!(unhedged_returns, w=2, label="Unhedged",c="#00467F")
    density!(objectives, w=2, label="Price Only",c="red")
    density!(gr_hedged, w=2, label="Price + Growth",c="green")
    density!(ev_hedged, w=2, label="Price + Evapotranspiration",c="orange")
    density!(hedged_returns, w=2, label="Price + Rainfall",c="blue")
    
    savefig(modelone("distribution.pdf"))
    
    plt
end

plot(
    plottransformeddistribution2(neutral, # xlims=(0, 1000),
        titlecode='a', titlestring="Risk Neutral"),
    plottransformeddistribution2(averse, # xlims=(0, 1000),
        titlecode='b', titlestring="Risk Averse: No Contracting"),
    plottransformeddistribution2(averse_sell, # xlims=(0, 1000),
        titlecode='c', titlestring="Risk Averse: Contracting"),
    layout=(1, 3), size=(1500,700)
)
savefig(modelone("heuristic_distribution2.pdf"))

In [ ]:
putpayoff(strike_price, realized_price) = max(0.0, strike_price - realized_price)
costput(strike_price, returns) = mean(putpayoff.(strike_price, returns))
chakravarti_delta(rainfall, threshold, payout) = rainfall < threshold ? payout : 0.0

function chakravarti(dataset, h, unhedged_returns=getindex.(dataset, :objective); strike_ratio=0.35, return_proportion=1/3)
    rainfall = [sum(x[:rainfall][21:30]) for x in dataset]
    delta_omega = chakravarti_delta.(rainfall, strike_ratio * mean(rainfall), return_proportion * mean(unhedged_returns))
    c = mean(delta_omega)
    @show quantile(delta_omega, 0:0.1:1)
    return unhedged_returns + h * (delta_omega - c)
end

function option_hedge(dataset, beta, strike_prices)
    unhedged_returns = getindex.(dataset, :objective)
    prices = [clamp(x[:price][end], 3, 9) for x in dataset]

    call_costs = [costput(p, prices) for p in strike_prices] + 0.02

    # number of call options
    C = length(strike_prices)
    # number of observations
    J = length(unhedged_returns)
    m = Model(solver=ClpSolver())
    @variables(m, begin
        ξ           # VaR
        h[1:C] >= 0 # buys
        x[1:J]      # returns
        z[1:J] <= 0 # cvar return
    end)
    @objective(m, Min, ξ - sum(z) / beta / J)
    @constraint(m, [i=1:J],
        x[i] == unhedged_returns[i] +
            sum( (putpayoff(strike_prices[c], prices[i]) - call_costs[c]) * h[c] for c in 1:C)
    )
    @constraint(m, z .<= x + ξ)
    solve(m)
    hedged_returns = getvalue(x)
    put_buys = getvalue(h)
    return put_buys, unhedged_returns, hedged_returns
end

put_buys, unhedged_returns, hedged_returns = option_hedge(neutral, 0.25, [5])
put_buys2, unhedged_returns, hedged_returns2 = option_hedge(neutral, 0.25, [6])
# put_buys3, unhedged_returns, hedged_returns3 = option_hedge(neutral, 0.25, [7])
hedged_returns3 = chakravarti(neutral, 1, hedged_returns2)

# put_buys1, unhedged_returns1, futures_returns = option_hedge(neutral, 0.5, 3:0.25:8)

@show cvar(hedged_returns, 0.5)
@show cvar(hedged_returns2, 0.5)
@show cvar(hedged_returns3, 0.5)
prices = [clamp(x[:price][end], 3, 9) for x in neutral]

cvar_hedge_ratio = cvar_hedgeratio(unhedged_returns, prices, 0.25)
futures_returns = hedge(unhedged_returns, prices, cvar_hedge_ratio)
@show cvar(futures_returns, 0.5)

plot(
    size = (750, 450),
    yticks=false,
    xlabel="Operating Profit (\$/Ha)\n",
    ylabel="Smoothed Density",
    left_margin=2mm,
    legend=:topleft
)
density!(unhedged_returns, label="Unhedged", w=2,c="#00467F")
density!(futures_returns, label="Futures", w=2,c="green")
density!(hedged_returns, label="\$5 Option", w=2,c="red", alpha=0.3)
density!(hedged_returns2, label="\$6 Option", w=2,c="red", alpha=0.6)
density!(hedged_returns3, label="Chakravarti", w=2,c="blue")
savefig(modelone("options_contract.pdf"))

In [ ]:
580*0.02

In [ ]:
function mycor(dataset, f; kwargs...)
    plot(
#         ylims=(-1,1),
        xlabel="Stage\n",
        ylabel="Correlation Coefficient",
        left_margin=5mm,
        right_margin=5mm,
        bottom_margin=10mm,
        size=(500,300);
        kwargs...)
    plot!([cor(getindex.(dataset, :objective), f.(dataset, t)) for t in 1:52], legend=false, w=3, c="#00467F")
end

w,h =1,3
plot(
mycor(neutral, (x,t)->x[:rainfall][t], title="(a) Rainfall"),
mycor(neutral, (x,t)->x[:evapotranspiration][t], title="(b) Evapotranspiration"),
mycor(neutral, (x,t)->sum(x[:rainfall][i] for i in max(1,t-9):t), title="(c) Soil Moisture"),
    layout=(w,h), size=(500*h, 300*w)
)
savefig(modelone("heuristic_distribution.pdf"))

In [ ]:
obj = getindex.(neutral, :objective)
price = [x[:price][end] for x in neutral]
hedged_price = hedge(obj, price, minvar_hedgeratio(obj, price))
println("Price: $(cor(obj, price))")

println("Growth: $(cor(obj, [sum(x[:gr]) for x in neutral]))")
println("Growth: $(cor(hedged_price, [sum(x[:gr]) for x in neutral]))")
    
println("Evapotranspiration: $(cor(obj, [sum(x[:evapotranspiration]) for x in neutral]))")
println("Evapotranspiration: $(cor(hedged_price, [sum(x[:evapotranspiration]) for x in neutral]))")
    
println("Summer Rainfall: $(cor(obj, [sum(x[:rainfall][21:30]) for x in neutral]))")
println("Summer Rainfall: $(cor(hedged_price, [sum(x[:rainfall][21:30]) for x in neutral]))")
    
println("Summer Rainfall: $(cor(obj, log.([sum(x[:rainfall][21:30]) for x in neutral])))")
println("Summer Rainfall: $(cor(hedged_price, log.([sum(x[:rainfall][21:30]) for x in neutral])))")

In [ ]:
neutral = SDDP.load(modelone("NeutralWithContracting.results"));
averse = SDDP.load(modelone("AverseNoContracting.results"));
averse_sell = SDDP.load(modelone("AverseWithContracting.results"));

In [ ]:
q = [sum(n[:milk]) for n in averse]
p = [n[:price][end] for n in averse]

(mean(p.^2 .* q) - mean(p) * mean(p.*q)) / var(p), mean(q)
# scatter(price, milk, legend=false, size=(500, 300), alpha=0.3, markerstrokewidth=0)

In [ ]:
function processobj(data)
    obj = [n[:objective] - 3536 for n in data]
    println(median(obj))
    println(mean(obj))
    println(std(obj))
    println()
end
processobj(neutral)
processobj(averse)
processobj(low_averse_sell)
processobj(averse_sell)
processobj(high_averse_sell)

In [ ]:
# neutral = SDDP.load("NeutralWithContracting.results");
# averse = SDDP.load("AverseNoContracting.results");
# low_averse_sell = SDDP.load("LowAverseWithContracting.results");
# averse_sell = SDDP.load("AverseWithContracting.results");
# high_averse_sell = SDDP.load("HighAverseWithContracting.results");
# vlow_averse_sell = SDDP.load("VLowWithContracting.results")[1:2000];
# plotall(vlow_averse_sell)
# savefig("vlow_averse_selling.pdf")
# vvlow_averse_sell = SDDP.load("VVLowWithContracting.results")[1:2000];
# plotall(vvlow_averse_sell)
# savefig("vvlow_averse_selling.pdf")

In [ ]:
function getresults(results, key, scalefactor=1.0)
    Q = [0.01, 0.1, 0.5, 0.9, 0.99, 0.01, 0.99]
    g2 = hcat([scalefactor.*r[key] for r in results]...)
    g3 = hcat([quantile(g2[g,:], Q) for g in 1:size(g2, 1)]...)'
    DataFrame(week=1:size(g3, 1), x10=g3[:,1], x25=g3[:,2], x50=g3[:,3], x75=g3[:,4], x90=g3[:,5],x05=g3[:,6],x95=g3[:,7])
end

function plotpowder2(data, key, ylims, ylabel,title,scalefactor=1.0;plotline::Int=0, PLOT_BOTH = true)    
    if !PLOT_BOTH
        blue = getresults(data, key, scalefactor)
    else
        Q = quantile([a[:price][12][1] for a in data], [0.2, 0.8])
        @show Q
        blue = getresults([d for d in data if d[:price][12][1] <= Q[1]], key, scalefactor)
        red = getresults([d for d in data if d[:price][12][1] >= Q[2]], key, scalefactor)
    end
    
    plot()
    ribbonplot!(blue, :week, :x10, :x90, "#00467F", 0.3)
    ribbonplot!(blue, :week, :x25, :x75, "#00467F", 0.3)
    @df blue plot!(:week, :x50, c="#00467F", w=2)
    if plotline > 0
        plot!(scalefactor*data[plotline][key], c="red", w=3)
    end
                                    
    if PLOT_BOTH
        ribbonplot!(red, :week, :x10, :x90, "#e65100", 0.3)
        ribbonplot!(red, :week, :x25, :x75, "#e65100", 0.3)
        @df red plot!(:week, :x50, c="#e65100", w=2)
    end
    xticks = 1:4:24
    xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
    plot!(legend=false, ylims=ylims, ylabel=ylabel,title=title, xlims=(1,24), xticks=(xticks, xticklabels), xlabel="")
    plot!(size=(500,300))
end

function weeks(t::Int, T::Int=52)
    Tl = floor(Int, 52 / T)
    modulus = round(Int, T / (52 - Tl * T))
    offset = Tl * (t - 1) + floor(Int, (t-1) / modulus)
    if mod(t, modulus) == 0
        offset + (1:(Tl + 1))
    else
        offset + (1:Tl)
    end
end

function plotall2(data; plotprice=false,plotline=0)
    T = 24
    for d in data
        d[:contracted_proportion] = fill(0.0, T)
        d[:milk_production] = fill(0.0, T)
        d[:milk_sold] = fill(0.0, T)
        for t in 1:T
            d[:milk_production][t] = sum(d[:milk][1:t]) 
            d[:milk_sold][t] = sum(d[:milk_sales][1:t]) 
            d[:contracted_proportion][t] = d[:milk_sold][t] ./ sum(d[:milk])
        end
    end
    if !plotprice
        plot(
            plotpowder2(data, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(a)",plotline=plotline),
            plotpowder2(data, :P, (0,3500), "Pasture Cover\n(kg/Ha)","(b)",plotline=plotline),
            plotpowder2(data, :b, (0,6), "Palm Kernel Fed\n(kg/Cow/Day)", "(c)", 1 / 3 / 7 ./ length.(weeks.(1:T, T)) ,plotline=plotline),
            plotpowder2(data, :milk_production, (0,2000), "Milk Production\n(kg)", "(d)",plotline=plotline),
            plotpowder2(data, :milk_sold, (0,2000), "Milk Sales\n(kg)", "(e)",plotline=plotline),
            plotpowder2(data, :contracted_proportion, (0, 1.25), "Proportion of Milk Contracted\n(fraction)", "(f)", plotline=plotline),
            layout=(2,3), size=(1500,600)
        )
    else
        parameters = JSON.parsefile(modeltwo("model.parameters.json"))
        for n in data
            n[:gdt] = [p[1] for p in n[:price]]
            n[:accumulated] = [p[2] for p in n[:price]]
            n[:futures] = zeros(Float64, T)
            for stage in 1:T
                n[:futures][stage] = n[:accumulated][stage] + futures_contribution(n[:gdt][stage], parameters["sales_curve"], stage)
            end
        end
        plot(
            plotpowder2(data, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(a)",plotline=plotline),
            plotpowder2(data, :P, (0,3500), "Pasture Cover\n(kg/Ha)","(b)",plotline=plotline),
            plotpowder2(data, :b, (0,6), "Palm Kernel Fed\n(kg/Cow/Day)", "(c)", 1 / 3 / 7 ./ length.(weeks.(1:T, T)),plotline=plotline),
            plotpowder2(data, :milk_production, (0,2000), "Milk Production\n(kg)", "(d)",plotline=plotline),
            plotpowder2(data, :milk_sold, (0,2000), "Milk Sales\n(kg)", "(e)",plotline=plotline),
            plotpowder2(data, :contracted_proportion, (0, 1.5), "Proportion of Milk Contracted\n(fraction)", "(f)", plotline=plotline),
            plotpowder2(data, :gdt, (0, 12), "Spot Price\n(\$/kg)", "(g)",plotline=plotline),
            plotpowder2(data, :accumulated, (0, 10), "Accumulated Price\n(\$/kg)", "(h)",plotline=plotline),
            plotpowder2(data, :futures, (3, 9), "Futures Price\n(\$/kg)", "(i)",plotline=plotline),
            layout=(3,3), size=(1500,900)
        )
    end
end

function futures_contribution(gt, w, auction)
    y = 0.0
    for i in (auction+1):length(w)
        y += w[i] * (0.97 ^ (i - auction) * gt + 0.15 * sum(0.97^(j-1) for j in 1:(i-auction)))
    end
    y
end

In [ ]:
modeltwo(s) = joinpath("model_two", s)
averse2 = SDDP.load(modeltwo("AverseWithContracting.results"));
parameters = JSON.parsefile(modeltwo("model.parameters.json"))
println("Selling")
plotall2(averse2, plotprice=true)
savefig(modeltwo("averse_selling_two.pdf"))

println("Noises")
parameters = JSON.parsefile(modeltwo("model.parameters.json"))
for n in averse2
    n[:gdt] = [p[1] for p in n[:price]]
    n[:accumulated] = [p[2] for p in n[:price]]
    n[:futures] = zeros(Float64, 24)
    for stage in 1:24
        n[:futures][stage] = n[:accumulated][stage] + futures_contribution(n[:gdt][stage], parameters["sales_curve"], stage)
    end
end

plot(
    plotpowder2(averse2, :gdt, (0, 10), "Spot Price\n(\$/kg)", "(a)",plotline=828, PLOT_BOTH=false),
    plotpowder2(averse2, :accumulated, (0, 10), "Accumulated Price\n(\$/kg)", "(b)",plotline=828, PLOT_BOTH=false),
    plotpowder2(averse2, :futures, (0, 10), "Futures Price\n(\$/kg)", "(c)",plotline=828, PLOT_BOTH=false),
    layout=(1,3), size=(1500,300)
)
savefig(modeltwo("model_two_noise.pdf"))

In [ ]:
minimum([sum(a[:stageobjective]) for a in averse2])
maximum([sum(a[:Δ][1]) for a in averse2])

In [ ]:
quantile([n[:accumulated][end] for n in averse2], [0.0, 0.25, 0.5, 0.75, 1.0])
parameters = JSON.parsefile(modeltwo("model.parameters.json"))
quantile([
    sum(
        n[:milk_sales][t] * (
            n[:price][t][2] + futures_contribution(n[:price][t][1], parameters["sales_curve"], t)
        ) for t in 1:23
    ) + n[:price][end][2] * n[:M][end]
    for n in averse2
], [0.0, 0.25, 0.5, 0.75, 1.0])

In [ ]:
function cvar(x::Vector{Float64}, b::Float64)
    mean(x[x .<= quantile(x, b)])
end
function cvar(x::Vector{Dict{Symbol, Any}},b)
    cvar(Float64[s[:objective] for s in x], b)
end
function avar(x, λ, β)
   λ * cvar(x, 1.0) + (1 - λ) * cvar(x, β) 
end
@show avar(high_averse_sell, 0.25, 0.25)
@show avar(averse_sell, 0.25, 0.25)
@show avar(neutral, 0.25, 0.25)

@show avar(high_averse_sell, 0.5, 0.25)
@show avar(averse_sell, 0.5, 0.25)
@show avar(neutral, 0.5, 0.25)

In [ ]:
function meanobj(data)
    round(Int, median(Float64[s[:objective] for s in data])) - 3536
#     round(Int, cvar(Float64[s[:objective] for s in data] - 3536, 0.0))
end
meanobj(neutral), meanobj(averse), meanobj(low_averse_sell), meanobj(averse_sell), meanobj(high_averse_sell)

In [ ]:
function getmean(data)
    mean(sim[:objective] - 3536 for sim in data), std(sim[:objective] - 3536 for sim in data)
end
@show getmean(neutral)
@show getmean(averse)
@show getmean(averse_sell)
@show getmean(high_averse_sell)

In [ ]:
function undoformatting(x)
    if x[end] == 'K'
        return 1_000.0 * parse(Float64, String(x[1:(end-1)]))
    elseif x[end] == 'M'
        return 1_000_000.0 * parse(Float64, String(x[1:(end-1)]))
    else
        return parse(Float64, x)
    end 
end
# iteration, time, simulation, bound
function getlogdata(filename)
    y = open(filename, "r") do io
        y = Array{Float64}(0, 4)
        for i in 1:14
            readline(io)
        end
        while !eof(io)
            line = readline(io)
            if contains(line, "--")
                break
            end
            items = split(strip(line))
            if length(items) == 1
                break
            end
            z = undoformatting.(items[[4,8,1,2]])'
            y = vcat(y, z)
        end
        y
    end

    y[:, 1] = 1:size(y, 1)
    y
end

In [ ]:
function Plots.gr_legend_pos(s::Symbol, w::Float64, h::Float64)
    (Plots.viewport_plotarea[2] - 0.05 - w, Plots.viewport_plotarea[4] - 0.02)
end
rollingquantile(x, q) = [quantile(x[max(1, i-999):i], q) for i in 1:length(x)]
function plotribbon!(filename, column=3; kwargs...)
    A = getlogdata(filename)
    x = A[:, column]
    x -= 3536
    yu = rollingquantile(x, 0.9)
    yl = rollingquantile(x, 0.1)
    plot!(1:length(x), 0.5 * (yl + yu), ribbon=yu - yl; kwargs...)
end

scale=1.0
plot(
    ylims=(-2_000, 10_000),
    xlabel="Iteration\n",
    ylabel="Operating Profit\n(\$/Ha)",
    size=(500scale,300scale),
    right_margin=5mm,
    top_margin=5mm,
    bottom_margin=10mm,
    title="(a)",
    legend=:topright
)
plotribbon!(modeltwo("model_one_averse.log"), label="", alpha=1.0, color="#00467F", w=3)
plt1 = plotribbon!(modeltwo("model_two_averse.log"), label="", alpha=0.25, color="#e65100", w=3)

A1 = getlogdata(modeltwo("model_one_averse.log"))
A2 = getlogdata(modeltwo("model_two_averse.log"))
plot!(A1[:,4] - 3536, label="Model One", color="#00467F", w=3)
plt = plot!(A2[:,4] - 3536, label="Model Two", color="#e65100", w=3)

plot(
    xlabel="Milk Price (\$/kg)\n",
    ylabel="\nSmoothed Density",
    yticks=false,
    size=(500scale,300scale),
    title="(b)",
    bottom_margin=10mm,
    legend=false
)
averse1 = SDDP.load(modelone("AverseWithContracting.results"));
averse2 = SDDP.load(modeltwo("AverseWithContracting.results"));
density!([n[:price][end][1] for n in averse1], label="Model One", color="#00467F", w=3)
plt2 = density!([n[:price][end][2] for n in averse2], label="Model Two", color="#e65100", w=3)
       
plot(plt1, plt2, layout=(1,2), size=(1000, 350))
savefig(modeltwo("convergence.pdf"))

In [ ]:
using Distributions

struct Foo
    p::Distribution
    q::Distribution
end
X(c, f) = mean(f.p) * c + rand(f.p) * (rand(f.q) - c)

In [ ]:
P = Normal(1.5, 1)
Q = Beta(2)
f = Foo(P, Q)

var(X(mean(P), f))

In [ ]:
using Optim
function Y(c, seed)
    srand(seed)
    s = var(X(c[1], f) for i in 1:10_000)
    @show s
    s
end
